In [3]:
import pandas as pd
import os

In [ ]:
# Avergae GCP per district
# Number of at_risks students per district
# Engagement rate per district


# Extract District Information 
# 001 Washington District
# 002 New York District

# Create a table for districts
#distric_id
# district_name

In [59]:
# Define file paths
raw_data = '../data/raw/'
processed_data = '../data/processed/'



In [35]:
def create_district_mapping(filepath):
    filename = filepath.split('/')[-1].split(".")[0]

    district_mapping = {
        'school_alpha': 'Washington District',
        'school_beta': 'New York District',
        'school_gamma': 'New York District'
    }

     
    return district_mapping[filename]
    
    

    
    

In [15]:
def clean_data(file_path):

    df = pd.read_csv(file_path)
    df.columns = df.columns.str.strip().str.lower().str.replace(' ', '_')
    
    column_map = {
    'id': 'student_id',
    'full_name': 'full_name',
    'sex': 'gender',
    'maths': 'math_score',
    'english_language': 'english_score',
    'sci': 'science_score',
    'attendance_%': 'attendance_rate',
    'mathscore': 'math_score',
    'englishscore': 'english_score',
    'english_score': 'english_score',
    'science_score': 'science_score',
    'attendance': 'attendance_rate',
    'Sex': 'gender',
    'Maths': 'math_score'
}

    df.rename(columns=column_map, inplace=True)


    df['math_score'] = pd.to_numeric(df['math_score'], errors='coerce')
    df['english_score'] = pd.to_numeric(df['english_score'], errors='coerce')
    df['science_score'] = pd.to_numeric(df['science_score'], errors='coerce')
    df['attendance_rate'] = pd.to_numeric(df['attendance_rate'], errors='coerce')

    return df[['student_id', 'full_name', 'gender', 'math_score',
                'english_score', 'science_score', 'attendance_rate', 'term']]

In [58]:
file_names = ['school_alpha.csv', 'school_beta.csv', 'school_gamma.csv']
raw_files = [os.path.join(raw_data, f) for f in file_names]




In [60]:
more_data = []

for file in raw_files:
    clean_file = clean_data(file)
    clean_file['district'] = create_district_mapping(file)
    
    more_data.append(clean_file)
all_data = pd.concat(more_data, ignore_index=True)



 

In [ ]:
# exports new york and washington districts to their respective csv files 

nyc = all_data[all_data['district' ] == 'New York District']
dc = all_data[all_data['district' ] == 'Washington District']

nyc.to_csv(os.path.join(processed_data, '002_New_York_District.csv'), index=False)
dc.to_csv(os.path.join(processed_data, '001_Washington_District.csv'), index=False)

In [63]:
# exports all the data with districts to a single csv file 
all_data.to_csv(os.path.join(processed_data, 'schools_with_district.csv'), index=False)


In [78]:
at_risk = pd.read_csv(os.path.join(processed_data, 'at_risk_students.csv'))

In [80]:
mgdata = pd.merge(at_risk, all_data, on='student_id')

In [81]:
# number of at risk students per district 
def calculate_at_risk_students():
    
    at_risk_counts = mgdata.groupby('district')['risk_score_y'].count().reset_index()

    at_risk_counts.rename(columns={'risk_score_y': 'at_risk_students'}, inplace=True)

    return at_risk_counts
    
    

calculate_at_risk_students()

,district,at_risk_students
0,New York District,69
1,Washington District,110


In [82]:

mgdata

,student_id,full_name_x,gpa,risk_score_x,engagement_level,full_name_y,gender,math_score,english_score,science_score,attendance_rate,term,district,risk_score_y
0,5506-350,Mary Smith,59.000000,50,Low,Mary Smith,Male,92.0,14.0,71.0,74.98,2023_T1,Washington District,50.0
1,7873-325,Patricia Moore,39.000000,51,Low,Patricia Moore,Female,1.0,87.0,29.0,60.82,2024_T1,Washington District,50.0
2,3615-814,Robert Smith,47.333333,50,Low,Robert Smith,Female,63.0,59.0,20.0,68.49,2023_T2,Washington District,51.0
3,2584-467,Mary Miller,63.333333,50,Low,Mary Miller,Female,58.0,41.0,91.0,71.65,2024_T1,Washington District,50.0
4,2654-489,Mary Jones,19.333333,51,Low,Mary Jones,Female,38.0,17.0,3.0,66.82,2023_T2,Washington District,50.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
320,4522-955,Elizabeth Smith,53.666667,50,Low,Elizabeth Smith,Female,29.0,46.0,86.0,60.08,2023_T1,New York District,NaN
321,6828-505,Mary Jones,60.333333,50,Low,Mary Jones,Male,83.0,81.0,17.0,61.62,2024_T1,New York District,NaN
322,3848-131,Robert Brown,53.333333,50,Low,Robert Brown,Female,93.0,32.0,35.0,64.58,2023_T2,New York District,NaN
323,5093-196,William Moore,28.000000,51,Low,William Moore,Female,18.0,27.0,39.0,74.76,2023_T1,New York District,NaN
